<h1>Auto-completion voor juridische voetnoten</h1>

<b>Doel:</b> op basis van alleen ECLI een volledige en juist gestileerde juridische voetnoot teruggeven. Op dit moment ondersteund: HR + hoven en rechtbanken in Nederland en andere zoals CBB, EHRM, HvJ EU. Indien bekend wordt de vindplaats in de volgende tijdschriften meegenomen in de voetnoot:

      NJ, NJB, TvA, TvC, JOR, JOM, JM, AR, JONDR, JA, NJF

<i>NB: mist nog schuingedrukte zaakaanduiding tussen '(' en ')', maar wellicht sowieso handig om die zelf te onthouden bij een ECLI om te weten waar de uitspraak over gaat.</i>

<i>NB2: perfectie niet gegarandeerd.</i>

Usecase voor toekomst: van NJ naar ECLI (in de Asser wordt vaak alleen een NJ-nummer gegeven)


<h1>Import packages</h1>

In [ ]:

import urllib3
import xmltodict
import locale
import time
from datetime import datetime
locale.setlocale(locale.LC_ALL, 'nl_NL')
import re

import pandas as pd
import numpy as np


# Works only in jupyter notebook:
from IPython.core.display import display, HTML


<h1>Function definitions</h1>

In [ ]:
replace_list = {'Hoge Raad':'HR', 
                'met annotatie van':'m.nt.',
                'Gerechtshof':'Hof',
                'College van Beroep voor het bedrijfsleven':'CBb',
                r'([0-9]{1}) m.nt.':r'\1, m.nt.',
                r'<i>NJ</i> ([0-9]{4}), ([0-9]+),':r'<i>NJ</i> \1/\2,',
                'And':'&',
                'Rechtbank':'Rb',
                'Raad van State':'ABRvS'
               }


month_to_num = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06',
                'Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}


journal_abbeviations = ['NJ', 'NJB','TvA','TvC', 'JOR','JOM','JM','AR','JONDR','JA','NJF']

# FIXME: when do I want to load this?
echr_data = pd.read_csv('data/echr_2_0_0_structured_cases.csv')

def ecli_lookup(ecli,italicize_journals=True, preferred_journal='NJ'):
    # Remove whitespace
    ecli = "".join(ecli.split())
    
    # Split into elements
    
    ecli_elts = ecli.split(':')
    country_code = ecli_elts[1]
    court_code = ecli_elts[2]
    year = ecli_elts[3]
    case_code = ecli_elts[4]
    
    # Nederlandse rechtspraak (HR, Hoven, Rechtbanken)
    if (country_code == 'NL'):
        
        voetnoot = get_NL(ecli, preferred_journal)
   
    # Council of Europe (Raad van Europa, EHRM)
    if (country_code == 'CE'):
        voetnoot = get_CE(ecli)
        
    # European Union (HvJ EU)
    if (country_code == 'EU'):
        voetnoot = get_EU(ecli)
       
    # Replace some words you want replaced
    for van,naar in replace_list.items():
        voetnoot = re.sub(van, naar, voetnoot)

    if (italicize_journals):
        for journal_abbr in journal_abbeviations:
            voetnoot = voetnoot.replace(journal_abbr,'<i>'+journal_abbr+'</i>')

    return voetnoot

def get_CE(ecli):
    url_pre = 'https://hudoc.echr.coe.int/app/transform/rss?library=echreng&query=contentsitename:ECHR%20AND%20(NOT%20(doctype=PR%20OR%20doctype=HFCOMOLD%20OR%20doctype=HECOMOLD))%20AND%20((ecli:%22'
    url_post = '%22))&sort=&start=0&length=20&rankingModelId=11111111-0000-0000-0000-000000000000'
    url = url_pre + ecli + url_post
    print(url)
    http = urllib3.PoolManager()
    r = http.request('GET', url)
    data = r.data   
    data_xml = xmltodict.parse(data)
    list_of_results = data_xml['rss']['channel']['item']
    
    if (type(list_of_results)!=list):
        list_of_results = [list_of_results] # In this case, there is only 1 search result. Make it into a list.

    for item in list_of_results:

        court = 'EHRM'

        # Example case no: 40575/10;67474/10 - Chamber Judgment
        case_no = 'no. ' + item['description'].split(' - ')[0].replace(';',' en ')

        case_date = item['pubDate'] # Sample case date: Tue, 02 Oct 2018 00:00:00 GMT
        case_date = case_date.split(', ')[1] # Remove day of week
        case_date = "/".join([case_date.split(' ')[0], month_to_num[case_date.split(' ')[1]], case_date.split(' ')[2]])
        case_date = datetime.strptime(case_date, '%d/%m/%Y')
        case_date = case_date.strftime('%d %B %Y')
        case_date = " ".join([case_date.split(' ')[0].replace('0',''),case_date.split(' ')[1],case_date.split(' ')[2]])

        case_name = item['title']
        case_name = case_name.title()
        case_name = case_name.replace('V.','v.')
        case_name = case_name.replace('Case Of ','')
        case_name = '(<i>'+case_name+'</i>)'


        break  # TODO: possibly, use other search results as well?

    #voetnoot = ", ".join([court+' '+case_date, case_no, ecli + ' ' + case_name]) # Apparently no case no.?
    voetnoot = ", ".join([court+' '+case_date, ecli + ' ' + case_name])
    
    return voetnoot


def get_EU(ecli):
    url_pre = 'http://curia.europa.eu/juris/documents.jsf?&critereEcli='
    url_post ='&lgrec=nl&language=nl'
    url = url_pre + ecli + url_post
    print(url)

    http = urllib3.PoolManager()
    r = http.request('GET', url)
    data = r.data   
    case_name = re.findall(r'<td class="table_cell_nom_usuel">([^\<]+)</td>',str(data))[0] # Possible: take 1 of the results
    case_name = case_name.strip()
    case_name = case_name.title()
    case_name = case_name.replace('V.','v.')
    case_name = case_name.replace('Case Of ','')
    case_name = '(<i>'+case_name+'</i>)'

    case_no = re.findall(r'<td class="table_cell_aff">([^\<]+)</td>',str(data))[0]
    case_date = re.findall(r'<td class="table_cell_date">([^\<]+)</td>',str(data))[0]
    court = 'HvJ EU'

    case_date = datetime.strptime(case_date, '%d/%m/%Y')
    case_date = case_date.strftime('%d %B %Y')
    case_date = " ".join([case_date.split(' ')[0].replace('0',''),case_date.split(' ')[1],case_date.split(' ')[2]])

    voetnoot = ", ".join([court+' '+case_date, case_no, ecli + ' ' + case_name])
    
    return voetnoot

def get_NL(ecli, preferred_journal='NJ'):

    url = 'https://data.rechtspraak.nl/uitspraken/content?id='+ecli+'&return=META'
    print(url)
    http = urllib3.PoolManager()
    r = http.request('GET', url)
    data = r.data
    data = xmltodict.parse(data)
    
    ecli_elts = ecli.split(':')
    country_code = ecli_elts[1]
    court_code = ecli_elts[2]
    
    if (court_code == 'PHR'):
        print("Pas op: conclusies van de A-G worden nog niet ondersteund.")

    description_dict = data['open-rechtspraak']['rdf:RDF']['rdf:Description'][0]
    case_date = description_dict['dcterms:date']['#text']
    case_date = datetime.strptime(case_date, '%Y-%m-%d')
    case_date = case_date.strftime('%d %B %Y')
    case_date = " ".join([case_date.split(' ')[0].replace('0',''),case_date.split(' ')[1],case_date.split(' ')[2]])
    court = description_dict['dcterms:creator']['#text']

    vindplaatsen = list(description_dict['dcterms:hasVersion']['rdf:list'].values())
    if (type(vindplaatsen[0])==list):
        vindplaatsen = vindplaatsen[0] # Sometimes it is a list, while in this case there is only 1 value
    vindplaatsen.remove('Rechtspraak.nl') # Deze is niet zo interessant.
    
    if (len(vindplaatsen)>0):
        tmplist = [s.split(' ')[0] for s in vindplaatsen]
        if (preferred_journal in tmplist):
            vindplaatsen_nr = tmplist.index(preferred_journal)
        else:
            vindplaatsen_nr = -1  # FIXME: laat de gebruiker kiezen tussen vindplaatsen
        vindplaats = vindplaatsen[vindplaatsen_nr]
        
        # Maak journal schuingedrukt
        if (len(vindplaats.split(' '))>1):
            vindplaats = '<i>'+vindplaats.split(' ')[0]+'</i>'+' '+' '.join((np.array(vindplaats.split(' ')[1:]).flatten()))
    
    #### COMBINE INTO VOETNOOT

    if (len(vindplaatsen)>0):
        voetnoot = ", ".join([court+' '+case_date,ecli,vindplaats])
    else:
        voetnoot = ", ".join([court+' '+case_date,ecli])

    return voetnoot

<h1>Apply to input</h1>

In [ ]:
# Test-ECLIs
ecli = 'ECLI:CE:ECHR:2018: 1002JUD004057510'
ecli = 'ECLI:CE:ECHR:2006:0427JUD002794602' # This one has a date
ecli = 'ECLI:CE:ECHR:2005:1213JUD004359302' # this one has a date
#ecli = 'ECLI:NL:GHAMS:2020:802'
ecli = 'ECLI:NL:HR:2018:1800'
ecli = 'ECLI:NL:CBB:2020:801'
ecli = 'ECLI:EU:C:2015:772' # Tarcau
ecli = 'ECLI:EU:C:2016:700' # Dumitras
#ecli = 'ECLI:EU:C:2018:37' # Schrems
ecli = 'ECLI:EU:C:2018:320'
ecli = 'ECLI:CE:ECHR:2005:0726JUD007331601'
ecli = 'ECLI:CE:ECHR:2018: 1002JUD004057510'
ecli = 'ECLI:CE:ECHR:1992:0225JUD001080284' # Pfeifer en Plankl
ecli = 'ECLI:CE:ECHR:1999:0223DEC003173796' # Finse vent
'''ecli = 'ECLI:NL:RBAMS:2018:8299' 
ecli = 'ECLI:NL:RVS:2013:248'
ecli = 'ECLI:NL:RVS:2009:BH0479'
ecli = 'ECLI:NL:GHAMS:2015:2618'
ecli = 'ECLI:NL:HR:1965:AB7079' # Kelderluik
#ecli = 'ECLI:NL:RBMNE:2015:9661'
#ecli = 'ECLI:NL:RBZWB:2019:2849'
#ecli = 'ECLI:NL:GHAMS:2015:2043' # Test met NTHR
#ecli = 'ECLI:EU:C:2019:825' # Petrichova
#ecli = 'ECLI:NL:PHR:2018:788' # conclusie
'''
ecli = 'ECLI:EU:C:2016:700' # Dumitras
ecli = 'ECLI:NL:HR:2015:2463' 
ecli = 'ECLI:NL:HR:2019:268' # HBL/X
ecli = 'ECLI:NL:PHR:2018:1378'
ecli = 'ECLI:NL:HR:1994:ZC1488'
ecli = 'ECLI:NL:HR:2007:BA2511'
ecli = 'ECLI:NL:HR:2009:BJ0861' # Abn amro/hamm
ecli = 'ECLI:NL:HR:2019:1841' # X/Heijmans
ecli = 'ECLI:NL:HR:2012:BT6947' # 
ecli = 'ECLI:NL:HR:2019:268' # HBL/X
ecli = 'ECLI:NL:HR:2019:1841' # X/Heijmans
ecli = 'ECLI:NL:HR:2002:AE7842' 
ecli = 'ECLI:NL:HR:2009:BJ0861'

# TODO: ook roepnaam van het arrest bijvoegen

print('Input:',ecli,'\n')
result = ecli_lookup(ecli)
print('\n')
display(HTML(result))

Input: ECLI:NL:HR:2009:BJ0861 

https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:HR:2009:BJ0861&return=META


